In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import *
import scanpy as sc
import os
import pyBigWig
from multiprocessing import Pool, cpu_count
import anndata
import scprinter as scp

/home/rzhang/miniforge3/envs/seq2print/lib/python3.11/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/home/rzhang/miniforge3/envs/seq2print/lib/python3.11/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/home/rzhang/miniforge3/envs/seq2print/lib/python3.11/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/home/rzhang/miniforge3/envs/seq2print/lib/python3.11/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom inst

In [2]:
def process_bigwig(bigwig_file, bed_df):
    """
    Process a single BigWig file to compute average signals for each BED region.
    
    :param bigwig_file: Path to the BigWig file.
    :param bed_df: Pandas DataFrame with 'Chromosome', 'Start', and 'End' columns.
    :return: Numpy array of average signals for each region.
    """
    bw = pyBigWig.open(bigwig_file)
    avg_signals = []

    for i, row in bed_df.iterrows():
        chrom, start, end = row["Chromosome"], row["Start"], row["End"]
        if chrom not in scp.genome.mm10.chrom_sizes:
            continue
        end = min(end, scp.genome.mm10.chrom_sizes[chrom]-1)
        try:
            values = bw.values(chrom, start, end, numpy=True)
        except:
            print (chrom, start, end)
            raise EOFError
        # values = np.array(values, dtype=np.float32)
        # values = values[~np.isnan(values)]  # Remove NaNs
        # values = np.nan_to_num(values)
        avg_signals.append(np.nan_to_num(np.nansum(values)))

    bw.close()
    return avg_signals
def get_parallel_sum_bigwig_signals(bed_df, bigwig_files):
    """
    Compute the average signal from multiple BigWig files in parallel.

    :param bed_df: Pandas DataFrame with 'Chromosome', 'Start', and 'End' columns.
    :param bigwig_files: List of paths to BigWig files.
    :return: Numpy array of final averaged signals per region.
    """
    with Pool(min(len(bigwig_files), cpu_count())) as pool:
        results = pool.starmap(process_bigwig, [(bw, bed_df) for bw in bigwig_files])

    return np.array(results)

# to peak count matrix

In [3]:
work_dir = '/bap/rzhang/Surya_DSS_em'

regions = './DSS21BC-merge2-ATAC.cleanPeaks.fixedwidthpeaks_800bp_300reSized.bed'

import scprinter as scp
regions_original = scp.utils.regionparser(regions)

In [4]:
chroms = set(list(scp.genome.mm10.chrom_sizes.keys()))

In [5]:
samples = ['DSS21orgs_P4_M1',
'DSS21orgs_P4_M2',
'DSS21orgs_P4_M3',
'DSS21orgs_P4_M7',
'DSS21orgs_P4_M8',
'DSS21orgs_P4_M9']
extra = 'DSS21orgs'

samples = ['DSS41prim_M1',
'DSS41prim_M2',
'DSS41prim_M3',
'DSS41prim_M4',
'DSS41prim_M5',
'DSS41prim_M6',
'DSS41prim_M7',
'DSS41prim_M8',
'DSS41prim_M9',
'DSS41prim_M10',
'DSS41prim_M11',
'DSS41prim_M12']
extra = 'DSS41prim'

In [6]:

cov_bigwig_files = [f'{work_dir}/BedMethyl/{x}.markdup.sorted_CpG.coverage.bw' for x in samples]
mod_bigwig_files = [f'{work_dir}/BedMethyl/{x}.markdup.sorted_CpG.methyl_count.bw' for x in samples]
frac_bigwig_files = [f'{work_dir}/BedMethyl/{x}.markdup.sorted_CpG.methylfrac.bw' for x in samples]

In [8]:
cre_bin = 300
methyl_bin = 1000
regions = scp.utils.resize_bed_df(regions_original, width=methyl_bin)
cov = get_parallel_sum_bigwig_signals(regions, cov_bigwig_files)
mod = get_parallel_sum_bigwig_signals(regions, mod_bigwig_files)

In [9]:
adata_frac = anndata.AnnData(X=(mod / cov))
adata_frac.obs.index = samples
cre_bin_final = cre_bin
regions_cre = scp.utils.resize_bed_df(regions_original, width=cre_bin_final)
adata_frac.var = regions_cre
adata_frac.var.index = scp.utils.df2regionidentifier(regions_cre)
adata_frac.write('/bap/rzhang/Surya_DSS_em/DSS41.DSS21BC-merge2-ATAC.cleanPeaks.1kb.frac.h5ad')
frac_df = pd.DataFrame(adata_frac.X)
frac_df.index = adata_frac.obs.index
frac_df.columns = adata_frac.var.index
frac_df.to_csv('/bap/rzhang/Surya_DSS_em/DSS41.DSS21BC-merge2-ATAC.cleanPeaks.1kb.frac.txt.gz', sep='\t', index=True, header=True)

/tmp/ipykernel_24578/864947112.py:1: RuntimeWarning: invalid value encountered in divide
  adata_frac = anndata.AnnData(X=(mod / cov))
/home/rzhang/miniforge3/envs/seq2print/lib/python3.11/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
... storing 'Chromosome' as categorical


In [10]:
frac_df

,chr1:3011687-3011987,chr1:3046521-3046821,chr1:3047520-3047820,chr1:3048385-3048685,chr1:3113301-3113601,chr1:3164963-3165263,chr1:3202553-3202853,chr1:3203973-3204273,chr1:3208418-3208718,chr1:3249117-3249417,...,chrX:169982067-169982367,chrX:169984088-169984388,chrX:169986899-169987199,chrX:169994030-169994330,chrX:170004284-170004584,chrX:170005171-170005471,chrX:170672711-170673011,chrX:170673555-170673855,chrX:170674495-170674795,chrX:170676226-170676526
DSS41prim_M1,0.817391,0.688889,0.888889,0.534483,0.687500,0.852941,0.684211,0.360000,0.698225,0.690722,...,0.820056,0.810282,0.843053,0.781133,0.618812,0.648923,0.674757,0.685168,0.762467,0.925281
DSS41prim_M2,0.721311,0.617647,0.803571,0.675000,0.621622,0.756757,0.554054,0.366197,0.607143,0.572917,...,0.820690,0.834751,0.766423,0.754202,0.518106,0.537904,0.619910,0.738623,0.827035,0.906504
DSS41prim_M3,0.709677,0.600000,0.840000,0.650000,0.666667,0.857143,0.666667,0.545455,0.650000,0.536585,...,0.754157,0.791411,0.763019,0.728945,0.581006,0.567568,0.604167,0.720000,0.826176,0.885965
DSS41prim_M4,0.833333,0.634483,0.782609,0.431373,0.653179,0.827586,0.615385,0.411765,0.644737,0.616162,...,0.845895,0.869759,0.738679,0.738553,0.555556,0.564068,0.616541,0.709859,0.872289,0.945695
DSS41prim_M5,0.739726,0.586466,0.863636,0.530303,0.707143,0.833333,0.476190,0.409091,0.720497,0.602041,...,0.813084,0.826063,0.797092,0.802395,0.573620,0.575850,0.686636,0.757344,0.817473,0.913043
DSS41prim_M6,0.804598,0.589552,0.836066,0.470588,0.690141,0.951219,0.587500,0.357143,0.629870,0.580247,...,0.830639,0.820833,0.654452,0.677842,0.547297,0.478049,0.747508,0.759240,0.737785,0.933951
DSS41prim_M7,0.816327,0.657895,0.833333,0.487805,0.738255,0.903226,0.680000,0.526316,0.786765,0.645570,...,0.852289,0.846154,0.825878,0.777709,0.612000,0.604211,0.622449,0.761218,0.837624,0.954128
DSS41prim_M8,0.867347,0.675676,0.844828,0.600000,0.777027,0.823529,0.655738,0.553846,0.653846,0.569892,...,0.836390,0.820125,0.770284,0.786450,0.674095,0.625678,0.687764,0.706507,0.788690,0.926901
DSS41prim_M9,0.852174,0.633588,0.781250,0.608696,0.794444,0.833333,0.630137,0.402985,0.771429,0.704762,...,0.847173,0.834254,0.829898,0.786315,0.660985,0.619939,0.622517,0.760534,0.806373,0.934286
DSS41prim_M10,0.841121,0.581560,0.886792,0.500000,0.588235,0.806452,0.556962,0.341772,0.635762,0.591667,...,0.810455,0.827397,0.635700,0.675385,0.555804,0.495114,0.700521,0.723709,0.785872,0.912195


# single cpg table

In [59]:
cpg_index_all, cpg_num, cpg_coord = scp.genome.mm10.fetch_cpg_index()

In [60]:
def get_cpg_signal(genome, BIGWIG_PATH):
    import pyBigWig
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from tqdm import tqdm
    
    bw = pyBigWig.open(BIGWIG_PATH)
    chroms = list(genome.chrom_sizes.keys())
    # === Extract methylation around each TSS ===
    profile = []
    location = []
    for chrom in tqdm(chroms):
        signal = []
        vv = bw.values(chrom, 0, genome.chrom_sizes[chrom], numpy=True)
        coord = np.array(cpg_coord[chrom])

        # for kk in [-2, -1, 0, 1, 2]:
        #     print (np.sum(np.isnan(vv[coord+kk])))
        signal.append(vv[coord])
        signal.append(vv[coord-1])
        # signal.append(vv[coord+2])
        # for ww in signal:
        #     print(np.sum(np.isnan(ww)))
        signal = np.nansum(signal, axis=0)
        profile.append(signal)
        location.append(pd.DataFrame({
                                     'start':coord-1,
                                     'end':coord}))
        location[-1]['chrom'] = chrom
    location = pd.concat(location, axis=0)
    bw.close()
    
    # === Compute average methylation profile ===
    
    # norm_factor = np.nansum(methylation_matrix[:, :100]) + np.nansum(methylation_matrix[:, -100:])
    # methylation_matrix = methylation_matrix / norm_factor[:, None]
    # print (np.sum(np.isnan(methylation_matrix)))
    profile = np.concatenate(profile)
    return profile, location

In [61]:
import bioframe
samples = ['DSS21orgs_P4_M1',
'DSS21orgs_P4_M2',
'DSS21orgs_P4_M3',
'DSS21orgs_P4_M7',
'DSS21orgs_P4_M8',
'DSS21orgs_P4_M9']
extra = 'DSS21orgs'
sample_grouping = {'DSS21orgs_P4_M1':'DSS',
'DSS21orgs_P4_M2':'DSS',
'DSS21orgs_P4_M3':'DSS',
'DSS21orgs_P4_M7':'control',
'DSS21orgs_P4_M8':'control',
'DSS21orgs_P4_M9':'control'}


samples = ['DSS41prim_M1',
'DSS41prim_M2',
'DSS41prim_M3',
'DSS41prim_M4',
'DSS41prim_M5',
'DSS41prim_M6',
'DSS41prim_M7',
'DSS41prim_M8',
'DSS41prim_M9',
'DSS41prim_M10',
'DSS41prim_M11',
'DSS41prim_M12']
extra = 'DSS41prim'
sample_grouping = {'DSS41prim_M1':'DSS_DMSO',
'DSS41prim_M2':'DSS_DMSO',
'DSS41prim_M3':'DSS_T5224',
'DSS41prim_M4':'DSS_DMSO',
'DSS41prim_M5':'DSS_T5224',
'DSS41prim_M6':'DSS_T5224',
'DSS41prim_M7':'Control_DMSO',
'DSS41prim_M8':'Control_DMSO',
'DSS41prim_M9':'Control_DMSO',
'DSS41prim_M10':'Control_T5224',
'DSS41prim_M11':'Control_T5224',
'DSS41prim_M12':'Control_T5224'}

In [62]:
feats_dict = {}
for sample in samples:
    # group = sample_grouping[sample]
    group = sample
    if group not in feats_dict:
        feats_dict[group] = [0, 0]
    print (sample)
    feats2, location = get_cpg_signal(scp.genome.mm10, f'{work_dir}/BedMethyl/{sample}.markdup.sorted_CpG.coverage.bw', )
    feats1, location = get_cpg_signal(scp.genome.mm10, f'{work_dir}/BedMethyl/{sample}.markdup.sorted_CpG.methyl_count.bw')
    feats_dict[group][0] += feats1
    feats_dict[group][1] += feats2
    # break

DSS41prim_M1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:33<00:00,  1.61s/it]


DSS41prim_M2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:33<00:00,  1.60s/it]


DSS41prim_M3


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:34<00:00,  1.66s/it]


DSS41prim_M11


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:35<00:00,  1.68s/it]


DSS41prim_M12


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:36<00:00,  1.72s/it]


In [63]:
final_df = []
groups = []
for group in feats_dict:
    groups.append(group)
    final_df.append(feats_dict[group][0] / feats_dict[group][1])
final_df = pd.DataFrame(final_df, index=groups)

/tmp/ipykernel_2387476/461287584.py:5: RuntimeWarning: invalid value encountered in divide
  final_df.append(feats_dict[group][0] / feats_dict[group][1])


In [64]:
location = location[['chrom', 'start', 'end']]


In [65]:
final_df.shape, location.shape

((12, 21867550), (21867550, 3))

In [66]:
for i in range(len(final_df)):
    location[final_df.index[i]] = final_df.iloc[i].values

In [67]:
chroms = list(scp.genome.mm10.chrom_sizes)[:-1]
chroms

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chrX']

In [68]:
location = location[location['chrom'].isin(set(chroms))]

In [71]:
location

,chrom,start,end,DSS41prim_M1,DSS41prim_M2,DSS41prim_M3,DSS41prim_M4,DSS41prim_M5,DSS41prim_M6,DSS41prim_M7,DSS41prim_M8,DSS41prim_M9,DSS41prim_M10,DSS41prim_M11,DSS41prim_M12
0,chr1,3000826,3000827,0.862069,0.925926,1.000000,0.869565,0.972222,0.857143,0.840000,0.870968,0.921053,0.961538,0.947368,0.925926
1,chr1,3001006,3001007,0.857143,0.958333,0.812500,0.692308,0.914286,0.875000,0.958333,0.909091,0.916667,0.743590,0.933333,0.814815
2,chr1,3001017,3001018,0.793103,0.750000,0.666667,0.833333,0.843750,0.774194,1.000000,0.900000,0.735294,0.714286,0.800000,0.781250
3,chr1,3001276,3001277,0.733333,0.512195,0.700000,0.633333,0.709677,0.636364,0.761905,0.653846,0.612245,0.552632,0.647059,0.750000
4,chr1,3001628,3001629,0.720000,0.714286,0.750000,0.750000,0.931035,0.818182,0.692308,0.812500,0.774194,0.827586,0.869565,0.880952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969464,chrX,170880598,170880599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969465,chrX,170880674,170880675,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969466,chrX,170880697,170880698,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969467,chrX,170880713,170880714,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
location.to_csv(f'./{extra}_cpg_methyl_info.bed.gz', sep='\t', index=False)